Imports

In [16]:
import pandas as pd
import numpy as np

# misc
import datetime as dt
from pprint import pprint
from itertools import chain

# reddit crawler
import praw

# sentiment analysis
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, RegexpTokenizer # tokenize words
from nltk.corpus import stopwords

import nltk
import ssl

nltk.download('vader_lexicon') # get lexicons data
nltk.download('punkt') # for tokenizer
nltk.download('stopwords')

r = praw.Reddit(user_agent='DemApollus',
                client_id='KrnUePjuO_J50XuOWNIyRw',
                client_secret='OswzYFW1nBcQWSfZPQVHmOXRglUcvw',
                check_for_async=False)

[nltk_data] Error loading vader_lexicon: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [17]:
sid = SentimentIntensityAnalyzer()

pos_text = "please take this class if u dont want to miss out on death!"
cap_pos_text = "Vader is AWESOME!" # captilization and ! increases the effect
neg_text = "Vader is bad"

print(sid.polarity_scores(pos_text))
print(sid.polarity_scores(cap_pos_text))
print(sid.polarity_scores(neg_text))

{'neg': 0.316, 'neu': 0.466, 'pos': 0.218, 'compound': -0.396}
{'neg': 0.0, 'neu': 0.281, 'pos': 0.719, 'compound': 0.729}
{'neg': 0.636, 'neu': 0.364, 'pos': 0.0, 'compound': -0.5423}


In [18]:
ubc_reddit = r.subreddit("UBC")
titles = list(())
urls = list(())
comments = list(())
for post in ubc_reddit.search("CPSC 121", limit=2):
    titles.append(post.title)
    urls.append(post.url)
    submission = r.submission(id=post.id)
    for comment in submission.comments.list():
        #print(post.title)
        comments.append(comment.body)
print(urls[0])

https://www.reddit.com/r/UBC/comments/ubreai/cpsc_121_final/


In [19]:
df = pd.DataFrame(comments)
df

,0
0,I totally failed because I didn’t study the ri...
1,I would cry but it’s my last final so I’m not ...
2,I'll see you all in the summer
3,"I’m a SD student from last term and well, look..."
4,Does anyone who took it before know if there w...
...,...
86,I wouldn't look to Japan of all places for imp...
87,Japan has its own problems. Do some research o...
88,"That sounds about right, I have a buddy at Bla..."
89,"Lmao so true, i wished id knew about this befo..."
